# 6 WorkFlow Gerencial, futuro=JULIO - Multiple Seeds Version + Feature Engineering Histórico Avanzado

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer
Modificado para ejecutar con 5 semillas diferentes

**NUEVA VERSIÓN CON FEATURE ENGINEERING HISTÓRICO AVANZADO**

Se incluyen las siguientes técnicas de feature engineering histórico:
1. **Lags múltiples**: Variables con retraso de 1, 2, 3 y 6 meses
2. **Deltas**: Diferencias entre períodos (delta1, delta2, delta3)
3. **Ventanas móviles**: Promedios, máximos, mínimos y desviaciones estándar en ventanas de 3 y 6 meses
4. **Tendencias**: Cálculo de tendencias lineales sobre ventanas temporales
5. **Ratios históricos**: Comparación del valor actual vs. promedios históricos
6. **Volatilidad**: Medidas de variabilidad temporal de las variables

#### 6.2 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
* Crear las carpetas en el Google Drive
* "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria kaggle de Python
* Bajar el **dataset_pequeno** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab
* Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget $webfiles/$archivo -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp $destino_bucket/$archivo $destino_local/$archivo
fi

## 6.3 Workflow con 5 Semillas

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

options(warn = -1)


#### Parametros Globales
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM_GLOBAL <- list()
PARAM_GLOBAL$experimento_base <- 6150  # Nuevo número de experimento para diferenciarlo
PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"

# Vector de 5 semillas diferentes
PARAM_GLOBAL$semillas <- c(153929, 838969, 922081, 795581, 194609)

# Lista para almacenar resultados de todas las semillas
resultados_totales <- list()

## 6.3 Loop Principal - Iteración Automática sobre las 5 Semillas

Este loop ejecuta todo el workflow completo para cada una de las 5 semillas de manera automática.

In [ ]:
# Loop automático sobre todas las semillas
for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

  cat("\n\n========================================\n")
  cat("PROCESANDO SEMILLA ", seed_idx, " de ", length(PARAM_GLOBAL$semillas), "\n")
  cat("Semilla: ", PARAM_GLOBAL$semillas[seed_idx], "\n")
  cat("========================================\n\n")

  # Inicializar PARAM para esta semilla
  PARAM <- list()
  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]
  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
  PARAM$dataset <- PARAM_GLOBAL$dataset
  PARAM$out <- list()
  PARAM$out$lgbm <- list()

  # ===================================================================
  # Carpeta del Experimento
  # ===================================================================
  setwd("/content/buckets/b1/exp")
  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx, "_FE_historico")
  dir.create(experimento_folder, showWarnings=FALSE)
  setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

  cat("Carpeta de trabajo: ", experimento_folder, "\n")

  # ===================================================================
  # Preprocesamiento del dataset
  # ===================================================================

  # Lectura del dataset
  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

  # Catastrophe Analysis
  dataset[ foto_mes==202006, internet:=NA]
  dataset[ foto_mes==202006, mrentabilidad:=NA]
  dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
  dataset[ foto_mes==202006, mcomisiones:=NA]
  dataset[ foto_mes==202006, mactivos_margen:=NA]
  dataset[ foto_mes==202006, mpasivos_margen:=NA]
  dataset[ foto_mes==202006, mcuentas_saldo:=NA]
  dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
  dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
  dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
  dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
  dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
  dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

  # Data Drifting - sin codigo en esta primera version

  # ===================================================================
  # Feature Engineering INTRA-MES
  # ===================================================================

  cat("\nIniciando Feature Engineering INTRA-MES...\n")

  atributos_presentes <- function( patributos )
  {
    atributos <- unique( patributos )
    comun <- intersect( atributos, colnames(dataset) )
    return( length( atributos ) == length( comun ) )
  }

  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  cat("Feature Engineering INTRA-MES completado.\n")

  # ===================================================================
  # Feature Engineering HISTÓRICO AVANZADO
  # ===================================================================

  cat("\n========================================\n")
  cat("INICIANDO FEATURE ENGINEERING HISTÓRICO AVANZADO\n")
  cat("========================================\n\n")

  # Ordenar dataset por cliente y mes (fundamental para operaciones de lag)
  setorder(dataset, numero_de_cliente, foto_mes)

  # Definir las columnas sobre las que se aplicará el feature engineering histórico
  # Excluimos: identificadores, variables temporales y target
  cols_lagueables <- copy( setdiff(
      colnames(dataset),
      c("numero_de_cliente", "foto_mes", "clase_ternaria")
  ) )

  cat("Número de variables base para FE histórico:", length(cols_lagueables), "\n\n")

  # ===================================================================
  # TÉCNICA 1: LAGS MÚLTIPLES (1, 2, 3 y 6 meses)
  # ===================================================================
  # Justificación: Las variables rezagadas capturan el estado histórico del cliente.
  # Los diferentes horizontes temporales (1, 2, 3, 6 meses) permiten al modelo
  # identificar patrones de corto, mediano y largo plazo.
  # Por ejemplo:
  # - lag1: El mes inmediatamente anterior (tendencias recientes)
  # - lag2-3: Comportamiento de corto plazo
  # - lag6: Comportamiento semestral, patrones estacionales

  cat("[1/6] Generando LAGS MÚLTIPLES (1, 2, 3 y 6 meses)...\n")

  # Lag 1 mes
  dataset[,
      paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
      by = numero_de_cliente,
      .SDcols = cols_lagueables
  ]

  # Lag 2 meses
  dataset[,
      paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
      by = numero_de_cliente,
      .SDcols = cols_lagueables
  ]

  # Lag 3 meses
  dataset[,
      paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
      by = numero_de_cliente,
      .SDcols = cols_lagueables
  ]

  # Lag 6 meses (captura estacionalidad semestral)
  dataset[,
      paste0(cols_lagueables, "_lag6") := shift(.SD, 6, NA, "lag"),
      by = numero_de_cliente,
      .SDcols = cols_lagueables
  ]

  cat("   ✓ Lags generados: ", length(cols_lagueables) * 4, " nuevas variables\n\n")

  # ===================================================================
  # TÉCNICA 2: DELTAS (Diferencias entre períodos)
  # ===================================================================
  # Justificación: Los deltas capturan el CAMBIO o VARIACIÓN de las variables.
  # Mientras que los lags capturan valores absolutos históricos, los deltas
  # capturan la VELOCIDAD del cambio.
  # Un cliente con delta negativo en saldo puede indicar deterioro financiero.
  # delta1 = valor_actual - valor_hace_1_mes (cambio reciente)
  # delta2 = valor_actual - valor_hace_2_meses (cambio en ventana más amplia)
  # delta3 = valor_actual - valor_hace_3_meses (tendencia trimestral)

  cat("[2/6] Generando DELTAS (cambios entre períodos)...\n")

  for (vcol in cols_lagueables)
  {
      # Delta 1: cambio respecto al mes anterior
      dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]

      # Delta 2: cambio respecto a hace 2 meses
      dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]

      # Delta 3: cambio respecto a hace 3 meses
      dataset[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  }

  cat("   ✓ Deltas generados: ", length(cols_lagueables) * 3, " nuevas variables\n\n")

  # ===================================================================
  # TÉCNICA 3: VENTANAS MÓVILES (Rolling statistics)
  # ===================================================================
  # Justificación: Las ventanas móviles suavizan la volatilidad y capturan
  # tendencias de mediano plazo. Aplicamos:
  # - PROMEDIO: Tendencia central del comportamiento reciente
  # - MÁXIMO: Picos de actividad/consumo
  # - MÍNIMO: Valles de actividad/consumo
  # - DESVIACIÓN ESTÁNDAR: Volatilidad/estabilidad del comportamiento
  #
  # Ventanas de 3 y 6 meses capturan patrones trimestrales y semestrales.

  cat("[3/6] Generando VENTANAS MÓVILES (rolling stats)...\n")
  cat("   Esto puede tardar varios minutos...\n")

  for (vcol in cols_lagueables)
  {
      # Ventana de 3 meses
      # Promedio móvil 3 meses
      dataset[, paste0(vcol, "_roll3_mean") := frollmean(
          x = get(vcol),
          n = 3,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]

      # Máximo móvil 3 meses
      dataset[, paste0(vcol, "_roll3_max") := frollapply(
          x = get(vcol),
          n = 3,
          FUN = max,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]

      # Mínimo móvil 3 meses
      dataset[, paste0(vcol, "_roll3_min") := frollapply(
          x = get(vcol),
          n = 3,
          FUN = min,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]

      # Desviación estándar móvil 3 meses (mide volatilidad)
      dataset[, paste0(vcol, "_roll3_sd") := frollapply(
          x = get(vcol),
          n = 3,
          FUN = sd,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]

      # Ventana de 6 meses
      # Promedio móvil 6 meses
      dataset[, paste0(vcol, "_roll6_mean") := frollmean(
          x = get(vcol),
          n = 6,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]

      # Desviación estándar móvil 6 meses
      dataset[, paste0(vcol, "_roll6_sd") := frollapply(
          x = get(vcol),
          n = 6,
          FUN = sd,
          align = "right",
          na.rm = TRUE
      ), by = numero_de_cliente]
  }

  cat("   ✓ Ventanas móviles generadas: ", length(cols_lagueables) * 6, " nuevas variables\n\n")

  # ===================================================================
  # TÉCNICA 4: TENDENCIAS (Slope/pendiente de regresión lineal)
  # ===================================================================
  # Justificación: La tendencia captura la DIRECCIÓN del cambio en el tiempo.
  # Calculamos la pendiente de una regresión lineal sobre ventanas de 3 y 6 meses.
  # Una pendiente positiva indica crecimiento, negativa indica decrecimiento.
  # Esto es más robusto que simples diferencias porque considera toda la ventana.
  # Ejemplo: Un saldo con pendiente negativa constante es señal de deterioro.

  cat("[4/6] Generando TENDENCIAS (slopes)...\n")

  # Función para calcular pendiente de regresión lineal
  calc_slope <- function(y) {
      if(all(is.na(y))) return(NA)
      x <- 1:length(y)
      valid <- !is.na(y)
      if(sum(valid) < 2) return(NA)
      tryCatch({
          coef(lm(y[valid] ~ x[valid]))[2]
      }, error = function(e) NA)
  }

  for (vcol in cols_lagueables)
  {
      # Tendencia sobre 3 meses
      dataset[, paste0(vcol, "_trend3") := frollapply(
          x = get(vcol),
          n = 3,
          FUN = calc_slope,
          align = "right"
      ), by = numero_de_cliente]

      # Tendencia sobre 6 meses
      dataset[, paste0(vcol, "_trend6") := frollapply(
          x = get(vcol),
          n = 6,
          FUN = calc_slope,
          align = "right"
      ), by = numero_de_cliente]
  }

  cat("   ✓ Tendencias generadas: ", length(cols_lagueables) * 2, " nuevas variables\n\n")

  # ===================================================================
  # TÉCNICA 5: RATIOS HISTÓRICOS
  # ===================================================================
  # Justificación: Los ratios comparan el valor actual contra promedios históricos.
  # Esto normaliza las variables y captura comportamientos anómalos.
  # ratio_vs_roll3 = valor_actual / promedio_3meses
  # - ratio > 1: valor actual por encima del promedio (posible pico)
  # - ratio < 1: valor actual por debajo del promedio (posible caída)
  # - ratio ≈ 1: comportamiento estable

  cat("[5/6] Generando RATIOS HISTÓRICOS...\n")

  for (vcol in cols_lagueables)
  {
      # Ratio: valor actual / promedio 3 meses
      dataset[, paste0(vcol, "_ratio_vs_roll3") :=
          ifelse(get(paste0(vcol, "_roll3_mean")) != 0,
                 get(vcol) / get(paste0(vcol, "_roll3_mean")),
                 NA)]

      # Ratio: valor actual / promedio 6 meses
      dataset[, paste0(vcol, "_ratio_vs_roll6") :=
          ifelse(get(paste0(vcol, "_roll6_mean")) != 0,
                 get(vcol) / get(paste0(vcol, "_roll6_mean")),
                 NA)]

      # Ratio: valor actual / valor hace 6 meses (cambio semestral relativo)
      dataset[, paste0(vcol, "_ratio_vs_lag6") :=
          ifelse(get(paste0(vcol, "_lag6")) != 0,
                 get(vcol) / get(paste0(vcol, "_lag6")),
                 NA)]
  }

  cat("   ✓ Ratios generados: ", length(cols_lagueables) * 3, " nuevas variables\n\n")

  # ===================================================================
  # TÉCNICA 6: VOLATILIDAD Y ESTABILIDAD
  # ===================================================================
  # Justificación: Medimos la ESTABILIDAD del comportamiento del cliente.
  # - Coeficiente de variación (CV) = desviación / promedio
  #   Normaliza la volatilidad respecto al nivel promedio
  # - Rango = máximo - mínimo
  #   Captura la amplitud de variación
  # Clientes con alta volatilidad pueden ser más riesgosos o estar
  # experimentando cambios importantes en su situación financiera.

  cat("[6/6] Generando métricas de VOLATILIDAD Y ESTABILIDAD...\n")

  for (vcol in cols_lagueables)
  {
      # Coeficiente de variación sobre 3 meses
      # CV = desv_std / media (normaliza la volatilidad)
      dataset[, paste0(vcol, "_cv3") :=
          ifelse(get(paste0(vcol, "_roll3_mean")) != 0,
                 get(paste0(vcol, "_roll3_sd")) / abs(get(paste0(vcol, "_roll3_mean"))),
                 NA)]

      # Coeficiente de variación sobre 6 meses
      dataset[, paste0(vcol, "_cv6") :=
          ifelse(get(paste0(vcol, "_roll6_mean")) != 0,
                 get(paste0(vcol, "_roll6_sd")) / abs(get(paste0(vcol, "_roll6_mean"))),
                 NA)]

      # Rango (amplitud de variación) sobre 3 meses
      dataset[, paste0(vcol, "_range3") :=
          get(paste0(vcol, "_roll3_max")) - get(paste0(vcol, "_roll3_min"))]
  }

  cat("   ✓ Métricas de volatilidad generadas: ", length(cols_lagueables) * 3, " nuevas variables\n\n")

  # ===================================================================
  # RESUMEN DE FEATURE ENGINEERING HISTÓRICO
  # ===================================================================

  total_features_nuevas <- length(cols_lagueables) * (4 + 3 + 6 + 2 + 3 + 3)

  cat("\n========================================\n")
  cat("RESUMEN FEATURE ENGINEERING HISTÓRICO\n")
  cat("========================================\n")
  cat("Variables base:", length(cols_lagueables), "\n")
  cat("\nNuevas features generadas:\n")
  cat("  - Lags (1,2,3,6):", length(cols_lagueables) * 4, "\n")
  cat("  - Deltas (1,2,3):", length(cols_lagueables) * 3, "\n")
  cat("  - Rolling stats:", length(cols_lagueables) * 6, "\n")
  cat("  - Tendencias:", length(cols_lagueables) * 2, "\n")
  cat("  - Ratios:", length(cols_lagueables) * 3, "\n")
  cat("  - Volatilidad:", length(cols_lagueables) * 3, "\n")
  cat("\nTOTAL NUEVAS FEATURES:", total_features_nuevas, "\n")
  cat("TOTAL FEATURES EN DATASET:", ncol(dataset), "\n")
  cat("========================================\n\n")

  # ===================================================================
  # Modelado - Training Strategy
  # ===================================================================

  cat("Configurando Training Strategy...\n")

  PARAM$trainingstrategy <- list()
  PARAM$trainingstrategy$validate <- c(202105)

  PARAM$trainingstrategy$training <- c(
    202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  PARAM$trainingstrategy$training_pct <- 1.0
  PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

  dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

  campos_buenos <- copy( setdiff(
      colnames(dataset), c("clase_ternaria","clase01","azar"))
  )

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset[, azar:=runif(nrow(dataset))]

  dataset[, fold_train := foto_mes %in% PARAM$trainingstrategy$training &
      (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
       azar < PARAM$trainingstrategy$training_pct ) ]

  if( !require("lightgbm")) install.packages("lightgbm")
  require("lightgbm")

  dtrain <- lgb.Dataset(
    data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
    label= dataset[fold_train == TRUE, clase01],
    free_raw_data= TRUE
  )

  dvalidate <- lgb.Dataset(
    data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
    label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data= TRUE
  )

  # ===================================================================
  # Hyperparameter Tuning
  # ===================================================================

  cat("\nIniciando Hyperparameter Tuning...\n")

  if(!require("DiceKriging")) install.packages("DiceKriging")
  require("DiceKriging")

  if(!require("mlrMBO")) install.packages("mlrMBO")
  require("mlrMBO")

  PARAM$hipeparametertuning <- list()
  PARAM$hipeparametertuning$num_interations <- 10
  PARAM$lgbm <- list()

  PARAM$lgbm$param_fijos <- list(
    objective= "binary",
    metric= "auc",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    force_row_wise= TRUE,
    seed= PARAM$semilla_primigenia,
    max_bin= 31,
    learning_rate= 0.03,
    feature_fraction= 0.5,
    num_iterations= 2048,
    early_stopping_rounds= 200
  )

  PARAM$hipeparametertuning$hs <- makeParamSet(
    makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
    makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
  )

  EstimarGanancia_AUC_lightgbm <- function(x) {

    param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

    modelo_train <- lgb.train(
      data= dtrain,
      valids= list(valid = dvalidate),
      eval= "auc",
      param= param_completo,
      verbose= -100
    )

    AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]
    attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

    rm(modelo_train)
    gc(full= TRUE, verbose= FALSE)

    message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

    return(AUC)
  }

  configureMlr(show.learner.output = FALSE)

  obj.fun <- makeSingleObjectiveFunction(
      fn= EstimarGanancia_AUC_lightgbm,
      minimize= FALSE,
      noisy= FALSE,
      par.set= PARAM$hipeparametertuning$hs,
      has.simple.signature= FALSE
  )

  ctrl <- makeMBOControl(
      save.on.disk.at.time= 600,
      save.file.path= "HT.RDATA"
  )

  ctrl <- setMBOControlTermination(
      ctrl,
      iters= PARAM$hipeparametertuning$num_interations
  )

  ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

  surr.km <- makeLearner(
      "regr.km",
      predict.type= "se",
      covtype= "matern3_2",
      control= list(trace = TRUE)
  )

  if (!file.exists("HT.RDATA")) {
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
  } else {
    bayesiana_salida <- mboContinue("HT.RDATA")
  }

  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  setorder(tb_bayesiana, -y, -num_iterations)

  fwrite( tb_bayesiana,
    file="BO_log.txt",
    sep="\t"
  )

  PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
    1,
    setdiff(colnames(tb_bayesiana),
      c("y","dob","eol","error.message","exec.time","ei","error.model",
        "train.time","prop.type","propose.time","se","mean","iter")),
    with= FALSE
  ]

  print(PARAM$out$lgbm$mejores_hiperparametros)

  # ===================================================================
  # Produccion
  # ===================================================================

  cat("\nEntrenando modelo final...\n")

  PARAM$trainingstrategy$final_train <- c(
    202105, 202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

  dfinal_train <- lgb.Dataset(
    data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
    label= dataset[fold_final_train == TRUE, clase01],
    free_raw_data= TRUE
  )

  fijos <- copy(PARAM$lgbm$param_fijos)
  fijos$num_iterations <- NULL
  fijos$early_stopping_rounds <- NULL

  param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

  final_model <- lgb.train(
    data= dfinal_train,
    param= param_final,
    verbose= -100
  )

  lgb.save(final_model, "modelo.txt")

  tb_importancia <- as.data.table(lgb.importance(final_model))
  fwrite( tb_importancia,
    file= "impo.txt",
    sep= "\t"
  )

  # ===================================================================
  # Scoring
  # ===================================================================

  cat("\nGenerando predicciones...\n")

  PARAM$trainingstrategy$future <- c(202107)
  dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

  prediccion <- predict(
    final_model,
    data.matrix(dfuture[, campos_buenos, with= FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]

  fwrite(tb_prediccion,
    file= "prediccion.txt",
    sep= "\t"
  )

  # ===================================================================
  # Curva de Ganancia
  # ===================================================================

  tb_prediccion[, clase_ternaria := dfuture$clase_ternaria ]
  tb_prediccion[, ganancia := -3000.0 ]
  tb_prediccion[clase_ternaria=="BAJA+2", ganancia := 117000.0 ]

  setorder( tb_prediccion, -prob )
  tb_prediccion[, gan_acum := cumsum(ganancia)]

  tb_prediccion[,
    gan_suavizada := frollmean(
      x= gan_acum,
      n= 400,
      align= "center",
      na.rm= TRUE,
      hasNA= TRUE
    )
  ]

  resultado <- list()
  resultado$ganancia_suavizada_max <- max( tb_prediccion$gan_suavizada, na.rm=TRUE )
  options(digits= 8)
  resultado$envios <- which.max( tb_prediccion$gan_suavizada)
  resultado$semilla <- PARAM$semilla_primigenia
  resultado$seed_idx <- seed_idx

  print(resultado)

  fwrite( tb_prediccion,
    file= "ganancias.txt",
    sep= "\t"
  )

  tb_prediccion[, envios:= .I]

  pdf("curva_de_ganancia.pdf")

  plot(
    x= tb_prediccion$envios,
    y= tb_prediccion$gan_acum,
    type= "l",
    col= "gray",
    xlim= c(0, 6000),
    ylim= c(0, 8000000),
    main= paste0("Seed ", seed_idx, " (FE Hist) - Gan= ", as.integer(resultado$ganancia_suavizada_max), " envios= ", resultado$envios),
    xlab= "Envios",
    ylab= "Ganancia",
    panel.first= grid()
  )

  dev.off()

  if( !require("yaml")) install.packages("yaml")
  require("yaml")

  PARAM$resultado <- resultado

  write_yaml( PARAM, file="PARAM.yml")

  # ===================================================================
  # Guardar resultado y limpiar para siguiente iteración
  # ===================================================================

  if(!exists("resultados_totales")) resultados_totales <- list()
  resultados_totales[[seed_idx]] <- resultado

  rm(dataset, dtrain, dvalidate, dfinal_train, final_model, tb_prediccion)
  gc(full=TRUE, verbose=FALSE)

  cat("\n========================================\n")
  cat("Semilla ", seed_idx, " completada exitosamente\n")
  cat("Ganancia: ", resultado$ganancia_suavizada_max, "\n")
  cat("Envios: ", resultado$envios, "\n")
  cat("========================================\n\n")

} # Fin del loop sobre las semillas

cat("\n\n***************************************\n")
cat("TODAS LAS SEMILLAS PROCESADAS\n")
cat("***************************************\n")



PROCESANDO SEMILLA  1  de  5 
Semilla:  153929 

Carpeta de trabajo:  WF6150_seed1_FE_historico 

Iniciando Feature Engineering INTRA-MES...
Feature Engineering INTRA-MES completado.

INICIANDO FEATURE ENGINEERING HISTÓRICO AVANZADO

Número de variables base para FE histórico: 31 

[1/6] Generando LAGS MÚLTIPLES (1, 2, 3 y 6 meses)...
   ✓ Lags generados:  124  nuevas variables

[2/6] Generando DELTAS (cambios entre períodos)...
   ✓ Deltas generados:  93  nuevas variables

[3/6] Generando VENTANAS MÓVILES (rolling stats)...
   Esto puede tardar varios minutos...


Streaming output truncated to the last 5000 lines.
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...):
“no non-missing arguments to min; returning Inf”
Warning message in .Primitive("min")(c(NA_real_, NA_real_, NA_real_), ...)

## Resumen de Resultados de Todas las Semillas

In [ ]:
# Crear tabla resumen
setwd("/content/buckets/b1/exp")

tb_resumen <- data.table(
  seed_idx = sapply(resultados_totales, function(x) x$seed_idx),
  semilla = sapply(resultados_totales, function(x) x$semilla),
  ganancia = sapply(resultados_totales, function(x) x$ganancia_suavizada_max),
  envios = sapply(resultados_totales, function(x) x$envios)
)

# Agregar estadísticas
tb_resumen[, rank := rank(-ganancia)]

cat("\n\n========================================\n")
cat("RESUMEN FINAL DE LAS 5 SEMILLAS\n")
cat("(CON FEATURE ENGINEERING HISTÓRICO AVANZADO)\n")
cat("========================================\n\n")
print(tb_resumen)

cat("\nESTADÍSTICAS:\n")
cat("Ganancia promedio: ", mean(tb_resumen$ganancia), "\n")
cat("Ganancia máxima: ", max(tb_resumen$ganancia), "\n")
cat("Ganancia mínima: ", min(tb_resumen$ganancia), "\n")
cat("Desviación estándar: ", sd(tb_resumen$ganancia), "\n")
cat("Coeficiente de variación: ", sd(tb_resumen$ganancia)/mean(tb_resumen$ganancia)*100, "%\n")
cat("Mejor semilla: ", tb_resumen[rank==1, semilla], " (seed_idx ", tb_resumen[rank==1, seed_idx], ")\n")

# Guardar resumen
fwrite(tb_resumen,
  file=paste0("resumen_5_seeds_exp", PARAM_GLOBAL$experimento_base, "_FE_historico.txt"),
  sep="\t"
)

# Guardar objeto completo
saveRDS(resultados_totales,
  file=paste0("resultados_completos_exp", PARAM_GLOBAL$experimento_base, "_FE_historico.rds")
)

cat("\nArchivos guardados:\n")
cat("- resumen_5_seeds_exp", PARAM_GLOBAL$experimento_base, "_FE_historico.txt\n")
cat("- resultados_completos_exp", PARAM_GLOBAL$experimento_base, "_FE_historico.rds\n")
cat("\nCada semilla tiene su carpeta individual con resultados detallados.\n")

cat("\n========================================\n")
cat("FEATURE ENGINEERING HISTÓRICO APLICADO:\n")
cat("========================================\n")
cat("1. Lags múltiples (1,2,3,6 meses)\n")
cat("2. Deltas (diferencias temporales)\n")
cat("3. Ventanas móviles (mean, max, min, sd)\n")
cat("4. Tendencias (slopes lineales)\n")
cat("5. Ratios históricos\n")
cat("6. Métricas de volatilidad (CV, rango)\n")
cat("========================================\n")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")